In [4]:
import base64
import json
import requests
import time
import datetime as dt
import xml.etree.ElementTree as ET

In [2]:
# initialize firestore admin session... only run once per kernel or errors
import firebase_admin
from firebase_admin import credentials, firestore
cred = credentials.Certificate('creds.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

In [1088]:
# ============================================= STATS FUNCTION (http request)
#@functions_framework.http
def stats(request):
    if 'symbol' not in request:
        return json.dumps({'error': 'missing symbol parameter'})
    symbol = request['symbol']
    basic_info = db.collection('map_figi').document(symbol).get()
    if not basic_info.exists:
        return json.dumps({'error': f'symbol {symbol} not found in database'})
    basic_info = basic_info.to_dict()
    return backend({'symbol':symbol} | basic_info) # return statement is temporary for testing
    stype = basic_info['stype'] if 'stype' in basic_info else None
    if not stype:
        return json.dumps({'error':f'could not identify security type for security {symbol}'})
    stats_response = {}
    if stype=='FUND':
        fund_stats = db.collection('map_figi').document(symbol).collection('stats').document('fund_stats').get()
        if not fund_stats.exists:
            return json.dumps({'error':f'could not find fund stats for security {symbol}'})
        stats_response['fund_stats'] = fund_stats.to_dict()
        if 'eqty_wgt' in stats_response['fund_stats']:
            eqty_stats = db.collection('map_figi').document(symbol).collection('stats').document('eqty_stats').get()
            stats_response = stats_response | {'eqty_stats':eqty_stats.to_dict()} if eqty_stats.exists else stats_response
    elif stype=='EQTY':
         eqty_stats = db.collection('map_figi').document(symbol).collection('stats').document('eqty_stats').get()
         if not eqty_stats.exists:
            return json.dumps({'error':f'could not find equity stats for security {symbol}'})
         stats_response['eqty_stats'] = eqty_stats.to_dict()
    return json.dumps(stats_response)


# ============================================= BACKEND FUNCTION (pub/sub)


# generic function to search for company submissions at the SEC, return the filing id (accession number)
def get_latest_filing(sec_id, submission_type):
    # submission_type:
    #   NPORT-P = most recent portfolio investments report
    #   N-CEN   = most recent annual report for registered investment companies
    submissions_url = f'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={sec_id}&type={submission_type}&output=atom' 
    print(submissions_url)
    submissions_header = # removed personal info
    submissions_response = requests.get(url=submissions_url, headers=submissions_header)
    if submissions_response.status_code != 200:
        return None
    raw_str = submissions_response.content.decode()
    ns = {'subs': raw_str.split('xmlns="')[1].split('"')[0]}
    root = ET.fromstring(raw_str)
    return filing_no.text.replace('-','') if ET.iselement(filing_no:=root.find('subs:entry/subs:content/subs:accession-number', ns)) else None


# cleaning fund names... lots of edge cases- not live yet, still observing and adding to the lists
def fund_name_formatter(series_name, class_name):
    s = series_name.replace('(R)','')
    c = class_name.replace(s, '').strip()
    sc = f'{s} {c}'
    single_words = {'PORTFOLIO':'','CLASS':'', 'SHARES':'','(R)':'','INSTITUTIONAL':'Instl','INVESTOR':'Inv','GOVERNMENT':'Govt',
                    'MUNICIPAL':'Muni','INTERNATIONAL':'Intl', 'TELECOMMUNICATIONS':'Telecom', 'TECHNOLOGY':'Tech', 'BIOTECHNOLOGY':'Biotech', 
                    'JPX-NIKKEI':'JPX-Nikkei','SERVICE':'Svc','CORPORATE':'Corp','AGGREGATE':'Agg','INTERMEDIATE':'Interm','USD':'USD','EM':'EM',
                    'SMID':'SMID','LTIP':'LTIP','TSLA':'TSLA','ZERO':'ZERO', 'PGIM':'PGIM','DFA':'DFA','BBH':'BBH','SAI':'SAI','IBD':'IBD',
                    'SAI':'SAI','ESG':'ESG','FPA':'FPA','GMO':'GMO','EX':'ex','RAFI':'RAFI','INDEX':'Idx','OPPORTUNITY':'Oppty','BTC':'BTC',
                    'JPMORGAN':'JPMorgan','STOXX':'STOXX','ETF':'ETF','JPX':'JPX','QQQ':'QQQ','SPDR':'SPDR','MSCI':'MSCI','ACWI':'ACWI','CSI':'CSI',
                    'PHARMACEUTICALS':'Pharma'}
    multi_words = {'High Yield':'HY','Emerging Markets':'EM','Ex-':'ex-','Investment Grade':'IG','  ':' ',' - ':' ','Us ':'US ','Dow Jones':'DJ'}
    sc = ' '.join([w.title() if w.isupper() and len(w)>2 else w for w in sc.split()]).strip()
    sc = ' '.join([single_words[w.upper()] if w.upper() in single_words else w for w in sc.split()]).replace('  ',' ')
    for x,y in multi_words.items():
        sc = sc.replace(x,y)
    return sc.strip()


def parse_form_ncen(cik, acc_num):
    ncen_url = f'https://www.sec.gov/Archives/edgar/data/{cik}/{acc_num}/primary_doc.xml'
    ncen_header = # removed personal info
    print(ncen_url)
    ncen_response = requests.get(url=ncen_url, headers=ncen_header)
    if ncen_response.status_code != 200:
        return
    raw_str = ncen_response.content.decode()
    ns = {'ncen': raw_str.split('xmlns="')[1].split('"')[0]}
    root = ET.fromstring(raw_str)
    if root.find('ncen:headerData/ncen:filerInfo/ncen:investmentCompanyType',ns).text != 'N-1A': # will need to go back and fix to be able to handle S-6, etc.
        print('not N-1A')
        print(ncen_url)
        return
    ncen_data = {}
    for fund_series in root.findall('ncen:formData/ncen:managementInvestmentQuestionSeriesInfo/', ns):
        series_name = fund_series.find('ncen:mgmtInvFundName',ns).text
        series_id = fund_series.find('ncen:mgmtInvSeriesId',ns).text
        for fund_class in fund_series.find('ncen:sharesOutstandings',ns):
            class_id = fund_class.attrib['sharesOutstandingClassId']
            class_name = fund_class.attrib['sharesOutstandingClassName']
            class_ticker = fund_class.attrib['sharesOutstandingTickerSymbol']
            if class_ticker.isalnum():
                ncen_data[class_ticker] = {'cik': cik, 'sid': series_id, 'cid': class_id, 'name': fund_name_formatter(series_name, class_name), 'stype': 'FUND'}
    for found_ticker, found_data in ncen_data.items():
        print(found_data['name'])
        figi = db.collection('map_ticker').document(found_ticker).get()
        if not figi.exists:
            print('no figi', found_ticker)
            db.collection('pending_review').document(found_ticker).set({'problem': 'no figi'} | found_data)
            continue
        figi = figi.to_dict()['figi']
        current_data = db.collection('map_figi').document(figi).get()
        if not current_data.exists:
            db.collection('pending_review').document(figi).set({'problem': 'no data', 'ticker': found_ticker} | found_data)
            continue
        current_data = current_data.to_dict()
        try:
            if all([(current_data['cik']==int(found_data['cik']) or current_data['cik']==found_data['cik']), 
                    current_data['sid']==found_data['sid'], current_data['cid']==found_data['cid']]):
                # going to hold off on updating names live until I see some more edge cases
                print('good', found_ticker)
                db.collection('map_figi').document(figi).set({'stype':'FUND'}, merge=True)
            else:
                print('data conflict', found_ticker)
                db.collection('pending_review').document(figi).set({'problem': 'data conflict', 'ticker': found_ticker} | found_data)
        except:
            print('data conflict', found_ticker)
            db.collection('pending_review').document(figi).set({'problem': 'data conflict', 'ticker': found_ticker} | found_data)
    return

def parse_form_nport(cik, acc_num):
    print('parsing nport')
    nport_url = f'https://www.sec.gov/Archives/edgar/data/{cik}/{acc_num}/primary_doc.xml'
    print(nport_url)
    nport_header = # removed personal info
    nport_response = requests.get(url=nport_url, headers=nport_header)
    if nport_response.status_code != 200:
        return None
    raw_str = nport_response.content.decode()
    ns = {'nport': raw_str.split('xmlns="')[1].split('"')[0]}
    root = ET.fromstring(raw_str)

    # extract fund (series) info
    # many data points are optional, or conditional mandatory, so it is good to check it actually exists before throwing Nonetype errors
    nport_as_of = dt.datetime.strptime(root.find('nport:formData/nport:genInfo/nport:repPdDate', ns).text,'%Y-%m-%d').date()

    ret_block = root.findall('nport:formData/nport:fundInfo/nport:returnInfo/nport:monthlyTotReturns/nport:monthlyTotReturn',ns)
    shared_nport_classes = [cid.attrib['classId'] for cid in ret_block if 'classId' in cid.attrib]

    print(f'Filing as of {nport_as_of}\n')
    #print(f'Fund Total Assets: {fund_tot_assets}')
    #print(f'Fund Total Liabilities: {fund_tot_liabs}')
    #print(f'Fund Leverage: {fund_leverage:.2%}')
    #print(f'fund cash: ${fund_cash:.2f}')
    print(f'sibling funds found: {shared_nport_classes}\n')

    parsed_holdings = []
    for holding in root.findall('nport:formData/nport:invstOrSecs/',ns):
        extracted = [('isin',x.attrib['value']) if ET.iselement(x:=holding.find('nport:identifiers/nport:isin',ns)) else None,
                     ('ticker',x.attrib['value'].replace('/','.')) if ET.iselement(x:=holding.find('nport:identifiers/nport:ticker',ns)) else None, # '/' will break the firestore db
                     (('cusip',x.text) if x.text.isalnum() else None) if ET.iselement(x:=holding.find('nport:cusip',ns)) else None,
                     ('pos_val_usd',float(x.text)) if ET.iselement(x:=holding.find('nport:valUSD',ns)) else None,
                     ('pos_val_pct',round(float(x.text)/100,5)) if ET.iselement(x:=holding.find('nport:pctVal',ns)) else None,
                     ('pos_qty',float(x.text)) if ET.iselement(x:=holding.find('nport:balance',ns)) else None,
                     ('type_qty',x.text) if ET.iselement(x:=holding.find('nport:units',ns)) else None,
                     ('asset_cat',x.text) if ET.iselement(x:=holding.find('nport:assetCat',ns)) else None,
                     ('issuer_cat',x.text) if ET.iselement(x:=holding.find('nport:issuerCat',ns)) else None,
                     (('inv_country',x.text) if x.text.isalnum() else None) if ET.iselement(x:=holding.find('nport:invCountry',ns)) else None,
                     ('bond_info',{'mat_date':dt.datetime.strptime(debt_dps.find('nport:maturityDt',ns).text,'%Y-%m-%d').date(),
                                   'cpn_type':debt_dps.find('nport:couponKind',ns).text,
                                   'cpn_rate':float(debt_dps.find('nport:annualizedRt',ns).text)/100}) if ET.iselement(debt_dps:=holding.find('nport:debtSec',ns)) else None,
                     ('currency_info',{'code':x.attrib['curCd'],
                                       'rate': float(x.attrib['exchangeRt'])}) if ET.iselement(x:=holding.find('nport:currencyConditional',ns)) else None]
        holding_info = dict([dp for dp in extracted if dp])
        parsed_holdings.append(holding_info)

    return {'filing_date':nport_as_of, 'fund_siblings':shared_nport_classes, 'holdings':parsed_holdings}

def analyze_holdings(nport_data, curr_data):
    as_of = nport_data['filing_date']
    holdings = nport_data['holdings']
    # first, run through all holdings and try to identify
    #for h in holdings:



    return nport_data

def update_fund(figi, curr_data=None, do_nport=True, do_ncen=False):
    print(f'running update_fund on {figi}')
    #print(curr_data)
    primary_id = curr_data['sid'] if 'sid' in curr_data else curr_data['cik']
    if do_nport:
        nport_accession_number = get_latest_filing(primary_id, 'NPORT-P')
        if nport_accession_number:
            print(f'cik: {curr_data["cik"]}')
            print(f'acc_num: {nport_accession_number}')
            nport_data = parse_form_nport(curr_data['cik'], nport_accession_number)
            if nport_data:
                return nport_data
                #return analyze_holdings(nport_data, curr_data)
    if do_ncen:
        ncen_accession_number = get_latest_filing(primary_id, 'N-CEN')
        if ncen_accession_number:
            parse_form_ncen(curr_data['cik'], ncen_accession_number)
    
    return


def backend(cloud_event):
    #try:
    #    pubsub_data = json.loads(base64.b64decode(cloud_event.data["message"]["data"]).decode('UTF-8'))
    #except:
    #    print('failed at json load')
    #    return
    print('in backend')
    pubsub_data = cloud_event
    print(pubsub_data)
    if 'symbol' not in pubsub_data:
        print('missing symbol key')
        return
    symbol = pubsub_data['symbol']
    if 'stype' not in pubsub_data:
        if 'cid' in pubsub_data:
            stype = 'FUND' # assume its a fund if it has a classID... I think this is always true, but won't write it to the db until it's confirmed by the SEC calls
        else:
            # however, it can still be a fund if it doesn't have a CID, SPY for example... so eventually need a better solution
            print(f'missing stype for {json.dumps(pubsub_data)}')
            db.collection('pending_review').document(symbol).set({'stype':'MISSING'}, merge=True)
            return
    else:
        stype = pubsub_data['stype']
    stale_data_threshold = -1
    try:
        proceed_update = (dt.date.today()-dt.datetime.strptime(pubsub_data['as_of'],'%Y%m%d').date()).days > stale_data_threshold
    except:
        proceed_update = True
    if not proceed_update:
        print(f'data for {symbol} is up to date')
        return
    
    if stype=='FUND':
        return update_fund(symbol, curr_data=pubsub_data)
    
    return


In [1145]:
#stats({'symbol':'BBG001SFSRH5'})
#res = stats({'symbol':'BBG001T93S65'})
#stats({'symbol': 'BBG00Q9MPQH1'})
#res = stats({'symbol': 'BBG00RYR0PY0'})
#stats({'symbol':'BBG001S64NG1'})
#stats({'symbol':'BBG001TC6MC1'})
#nport_data = stats({'symbol':'BBG001SBZHS6'}) # SGIIX
nport_data = stats({'symbol':'BBG001T93S65'}) # POLIX
#nport_data = stats({'symbol':'BBG00Q9MPQH1'}) # AVUS
#nport_data = stats({'symbol':'BBG0073806T8'}) # BCGDX
#nport_data

in backend
{'symbol': 'BBG001T93S65', 'cik': '1388485', 'stype': 'FUND', 'cid': 'C000089998', 'sid': 'S000029264', 'ticker': 'POLIX', 'as_of': '20230930', 'name': 'Polen Growth Institutional'}
running update_fund on BBG001T93S65
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=S000029264&type=NPORT-P&output=atom
cik: 1388485
acc_num: 000175272423220151
parsing nport
https://www.sec.gov/Archives/edgar/data/1388485/000175272423220151/primary_doc.xml
Filing as of 2023-07-31

sibling funds found: ['C000089998', 'C000089997']



In [1192]:
def get_figi(all_jobs):
    jobs_per_req = 10
    wait_per_req = 2.5
    openfigi_url = 'https://api.openfigi.com/v3/mapping'
    openfigi_headers = {'Content-Type': 'text/json'}
    return_data = []
    i = 0
    while i < len(all_jobs)/jobs_per_req:
        job_slice = all_jobs[i*jobs_per_req:(i+1)*jobs_per_req]
        time.sleep(0 if i==0 else wait_per_req-min((dt.datetime.now()-last_call).seconds,wait_per_req))
        response = requests.post(url=openfigi_url, headers=openfigi_headers, json=job_slice)
        last_call = dt.datetime.now()
        if response.status_code == 200:
            for figi_in, figi_out in zip(job_slice, response.json()):
                if 'data' not in figi_out:
                    continue
                figi_out = figi_out['data']
                figi_list = list(set([f['shareClassFIGI'] for f in figi_out if f['shareClassFIGI'] is not None])) # only concerned about equities/funds right now... bonds will return an empty list
                figi = figi_list[0] if figi_list else None
                ticker_list = list(set([f['ticker'] for f in figi_out if f['exchCode']=='US']))
                ticker = ticker_list[0] if ticker_list else None
                return_data.append(figi_in | ({'figi':figi} if figi else {}) | ({'ticker':ticker} if ticker and figi_in['idType']!='TICKER' else {}))
        i += 1
    return return_data



def analyze_holdings(nport_data, curr_data=None):
    as_of = nport_data['filing_date']
    holdings = nport_data['holdings']

    # this is where further asset class functionality will be introduced over time... see NPORT documentation for full list of asset_cat/issuer_cat possible pairs
    stype_map = {('STIV','CORP'): 'BOND',  # Short-term investment vehicle + Corporate
                 ('STIV','UST'): 'BOND',   # Short-term investment vehicle + U.S. Treasury
                 ('STIV','USGA'): 'BOND',  # Short-term investment vehicle + U.S. government agency
                 ('STIV','USGSE'): 'BOND', # Short-term investment vehicle + U.S. government sponsored entity
                 ('STIV','MUN'): 'BOND',   # Short-term investment vehicle + Municipal
                 ('STIV','NUSS'): 'BOND',  # Short-term investment vehicle + Non U.S. sovereign 
                 ('STIV','RF'): 'FUND',    # Short-term investment vehicle + Registered fund
                 ('DBT','CORP'): 'BOND',   # Debt + Corporate
                 ('DBT','UST'): 'BOND',    # Debt + U.S. Treasury
                 ('DBT','USGA'): 'BOND',   # Debt + U.S. government agency
                 ('DBT','USGSE'): 'BOND',  # Debt + U.S. government sponsored entity
                 ('DBT','MUN'): 'BOND',    # Debt + Municipal
                 ('DBT','NUSS'): 'BOND',   # Debt + Non U.S. sovereign 
                 ('DBT','RF'): 'FUND',     # Debt + Registered fund
                 ('EC','CORP'): 'EQTY',    # Equity-common + Corporate
                 ('EC','RF'): 'FUND',      # Equity-common + Registered fund
                }
    
    stats_ready = [] # weight included in stats/analysis & security is already in database (potentially is already up-to-date) OR security will not go in database (matured bonds, derivatives/commodities eventually)
    try_identify = [] # weight included in stats/analysis & will attempt to identify via openfigi API before moving to stats_ready
    unknown_exclude = [] # weight NOT included in stats/analysis & will NOT attempt to identify - unknown security type or some problem
    pending_figi_jobs = [] # formatted openfigi API jobs, to be zipped with try_identify

    # sort holdings into their proper buckets
    for h in holdings:  
        if ('asset_cat' not in h) or ('issuer_cat' not in h): # looks technically possible to not have asset_cat according to NPORT documentation, but haven't seen it yet
            # NOTE - this is screening out holdings with the issuerConditional tag, which isn't being passed to this function yet, but contains potentially useful info
            unknown_exclude.append(h)
            continue
        h['stype'] = stype_map[s] if ((s:=(h['asset_cat'], h['issuer_cat'])) in stype_map) else None
        if not h['stype']:
            print((h['asset_cat'], h['issuer_cat']),' -> ', h['stype'])
            unknown_exclude.append(h)
            continue
        if (h['stype']=='BOND') and (h['bond_info']['mat_date'] < dt.date.today()):
            stats_ready.append(h)
            continue
        figi_job = {}
        if 'isin' in h:
            if (search_db:=db.collection('map_isin').document(h['isin']).get()).exists:
                h['figi'] = search_db.to_dict()['figi']
                stats_ready.append(h)
                continue
            else:
                figi_job['idType'] = 'ID_ISIN'
                figi_job['idValue'] = h['isin']
        if 'ticker' in h:
            if (search_db:=db.collection('map_ticker').document(h['ticker']).get()).exists:
                h['figi'] = search_db.to_dict()['figi']
                stats_ready.append(h)
                continue
            elif not figi_job:
                figi_job['idType'] = 'TICKER'
                figi_job['idValue'] = h['ticker']
        if (not figi_job) and ('cusip' in h):
            figi_job['idType'] = 'ID_CUSIP'
            figi_job['idValue'] = h['cusip']
        if not figi_job:
            unknown_exclude.append(h) # no ISIN, TICKER, or CUSIP found in NPORT filing... will add more options eventually
            continue
        try_identify.append(h)
        pending_figi_jobs.append(figi_job)

    # attempt to id the holdings that were not found in the firestore db
    identify_result = get_figi(pending_figi_jobs)
    for sec_info, query in zip(try_identify, pending_figi_jobs):
        new_data = [res for res in identify_result if res['idType']==query['idType'] and res['idValue']==query['idValue']]
        if len(new_data)!=1:
            stats_ready.append(sec_info) # wasn't able to assign a figi
            continue
        sec_info |= {'figi':new_data[0]['figi']} if 'figi' in new_data[0] else {}
        stats_ready.append(sec_info)

    # incorporate geography hierarchy (market, region, country) & filing_date
    with open('backend_metadata.json') as file:
        backend_metadata = json.load(file)
    stats_ready = [h | backend_metadata['geography'][h['inv_country'] if 'inv_country' in h and h['inv_country'] in backend_metadata['geography'] else 'ZZ'] | {'mark_date': nport_data['filing_date']} for h in stats_ready]

    # separate major asset classes
    fund_total_wgt = sum([h['pos_val_pct'] for h in holdings])
    holdings_eqty = [h for h in stats_ready if h['stype']=='EQTY']
    holdings_bond = [h for h in stats_ready if h['stype']=='BOND']
    holdings_fund = [h for h in stats_ready if h['stype']=='FUND']

    # do updates
    for eqty in holdings_eqty:
        if 'figi' in eqty:
            eqty = update_equity(eqty)


    # start consolidating holding stats to make the db payload
    # path = ...stats/FUND/allocation
    fund_allocation = {}
    fund_allocation |= {'FUND': sum([f['pos_val_pct'] for f in holdings_fund])} if len(holdings_fund) != 0 else {} # temporary - need to figure out how to seperate this out
    fund_allocation |= {'EQTY': sum([e['pos_val_pct'] for e in holdings_eqty])} if len(holdings_eqty) != 0 else {}
    fund_allocation |= {'BOND': sum([b['pos_val_pct'] for b in holdings_bond])} if len(holdings_bond) != 0 else {}
    fund_allocation |= {'EROR': sum([b['pos_val_pct'] for b in unknown_exclude])} if len(unknown_exclude) != 0 else {}
    fund_allocation |= {'CASH': cash_alloc} if (cash_alloc:=(fund_total_wgt-sum([v for v in fund_allocation.values()]))) != 0 else {}

    #print(('map_figi', 'this figi', 'stats', 'FUND', 'allocation'), fund_allocation)
    db_payloads = {'stats': {'FUND': fund_allocation}}

    # path = ...stats/EQTY/...
    if 'EQTY' in fund_allocation:
        eqty_geography_market = {market: sum([j['pos_val_pct'] for j in holdings_eqty if j['market']==market])/fund_allocation['EQTY'] for market in set([i['market'] for i in holdings_eqty])}
        eqty_geography_region = {region: sum([j['pos_val_pct'] for j in holdings_eqty if j['region']==region])/fund_allocation['EQTY'] for region in set([i['region'] for i in holdings_eqty])}
        eqty_geography_country = {country: sum([j['pos_val_pct'] for j in holdings_eqty if j['country']==country])/fund_allocation['EQTY'] for country in set([i['country'] for i in holdings_eqty])}

        #print(('map_figi', 'this figi', 'stats', 'EQTY', 'market'), eqty_geography_market)
        #print(('map_figi', 'this figi', 'stats', 'EQTY', 'region'), eqty_geography_region)
        #print(('map_figi', 'this figi', 'stats', 'EQTY', 'country'), eqty_geography_country)
        db_payloads['stats'] |= {'EQTY': {'market': eqty_geography_market,
                                          'region': eqty_geography_region,
                                          'country': eqty_geography_country}}

    # path = ...stats/BOND/...
    if 'BOND' in fund_allocation:
        bond_geography_market = {market: sum([j['pos_val_pct'] for j in holdings_bond if j['market']==market])/fund_allocation['BOND'] for market in set([i['market'] for i in holdings_bond])}
        bond_geography_region = {region: sum([j['pos_val_pct'] for j in holdings_bond if j['region']==region])/fund_allocation['BOND'] for region in set([i['region'] for i in holdings_bond])}
        bond_geography_country = {country: sum([j['pos_val_pct'] for j in holdings_bond if j['country']==country])/fund_allocation['BOND'] for country in set([i['country'] for i in holdings_bond])}
        
        #print(('map_figi', 'this figi', 'stats', 'EQTY', 'market'), bond_geography_market)
        #print(('map_figi', 'this figi', 'stats', 'EQTY', 'region'), bond_geography_region)
        #print(('map_figi', 'this figi', 'stats', 'EQTY', 'country'), bond_geography_country)
        db_payloads['stats'] |= {'BOND': {'market': bond_geography_market,
                                          'region': bond_geography_region,
                                          'country': bond_geography_country}}


    print(db_payloads)  
    return holdings_eqty



def update_equity(update_data):


    return update_data


holdings_eqty =  analyze_holdings(nport_data)



{'stats': {'FUND': {'FUND': 0.02874, 'EQTY': 0.9720300000000001}, 'EQTY': {'market': {'DM': 1.0}, 'region': {'EU': 0.037889777064493896, 'AM': 0.9621102229355061}, 'country': {'IE': 0.037889777064493896, 'US': 0.9621102229355061}}}}


In [1200]:
def update_equity(update_data):
    db_response_basic = db.collection('map_figi').document(update_data['figi']).get()
    if not db_response_basic.exists:
        print(update_data['figi'], ' does not exist in db')
        return update_data
    current_basic = db_response_basic.to_dict()
    try: proceed_update = dt.datetime.strptime(current_basic['as_of'],'%Y%m%d').date() < update_data['mark_date']
    except: proceed_update = True
    if not proceed_update:
        # get the current stats data
        return

    print(current_basic)


    return update_data


eqty = holdings_eqty[0]
update_equity(eqty)

{'cik': '1800', 'stype': 'EQTY', 'ticker': 'ABT', 'name': 'ABBOTT LABORATORIES'}


{'isin': 'US0028241000',
 'ticker': 'ABT',
 'cusip': '002824100',
 'pos_val_usd': 276371047.17,
 'pos_val_pct': 0.03524,
 'pos_qty': 2482449.0,
 'type_qty': 'NS',
 'asset_cat': 'EC',
 'issuer_cat': 'CORP',
 'inv_country': 'US',
 'stype': 'EQTY',
 'figi': 'BBG001S5N9M6',
 'market': 'DM',
 'region': 'AM',
 'country': 'US',
 'mark_date': datetime.date(2023, 7, 31)}

In [1199]:
d = '20231001'
dt.datetime.strptime(d,'%Y%m%d').date() < dt.date(2023,10,17)

True

In [ ]:
# BASIC INFO PATHS
# ---------------------------------------------------------
# map_figi/figi/name = 'name'
# map_figi/figi/ticker = 'TICKER'
# map_figi/figi/cik = '123456'
# map_figi/figi/sid = 'S123456789'
# map_figi/figi/cid = 'C123456789'
# map_figi/figi/as_of = '20231016'
# map_figi/figi/stype = ['FUND', 'EQTY', 'BOND']


# SECURITY STATISTICS PATHS
# ---------------------------------------------------------

# if stype==EQTY, then the following are required:
#
#       map_figi/figi/stats/EQTY/market = ['DM', 'EM']
#       map_figi/figi/stats/EQTY/region = ['AM', 'EU', 'ME', 'AP', 'AF']
#       map_figi/figi/stats/EQTY/country = 'US', 'GB, etc.
#       map_figi/figi/stats/EQTY/mkt_cap = 123456789
#       map_figi/figi/stats/EQTY/price_earn = 1.0
#       map_figi/figi/stats/EQTY/price_book = 1.0
#       map_figi/figi/stats/EQTY/price_sale = 1.0
#       map_figi/figi/stats/EQTY/price_cf = 1.0
#       map_figi/figi/stats/EQTY/div_yld = 0.01

# if stype==FUND, the paths get more complicated...
#
#   fund asset allocation: not all are required (should omit if zero weight), however, if an stype is included, then the corresponding document is required at the 'stats' level (except for CASH)
#   leaving this intentionally open right now, to allow for increasingly granular asset classifications in the future (e.g .../EQTY/US_large_value = 0.5)
#       map_figi/figi/stats/FUND/allocation/EQTY/wgt = 0.6 
#       map_figi/figi/stats/FUND/allocation/BOND/wgt = 0.3
#       map_figi/figi/stats/FUND/allocation/CASH/wgt = 0.05
#       map_figi/figi/stats/FUND/allocation/UNKN/wgt = 0.05

#       map_figi/figi/stats/EQTY/geography/market/DM
#       map_figi/figi/stats/EQTY/geography/market/EM
#       map_figi/figi/stats/EQTY/geography/market/FM
#       map_figi/figi/stats/EQTY/geography/market/ZZ



# ...


# map_figi/figi/stats/BOND

In [953]:
def testing_parse_form_nport(cik, acc_num):
    print('parsing form nport')
    nport_url = f'https://www.sec.gov/Archives/edgar/data/{cik}/{acc_num}/primary_doc.xml'
    print(nport_url)
    nport_header = # removed personal info
    nport_response = requests.get(url=nport_url, headers=nport_header)
    if nport_response.status_code != 200:
        return None
    raw_str = nport_response.content.decode()
    ns = {'nport': raw_str.split('xmlns="')[1].split('"')[0]}
    root = ET.fromstring(raw_str)

    # extract fund (series) info
    # many data points are optional, or conditional mandatory, so it is good to check it actually exists before throwing Nonetype errors
    nport_as_of = dt.datetime.strptime(root.find('nport:formData/nport:genInfo/nport:repPdDate', ns).text,'%Y-%m-%d').date()
    fund_tot_assets = float(root.find('nport:formData/nport:fundInfo/nport:totAssets',ns).text)
    fund_tot_liabs = float(root.find('nport:formData/nport:fundInfo/nport:totLiabs',ns).text)
    fund_leverage = fund_tot_assets / (fund_tot_assets - fund_tot_liabs)
    fund_cash = float(x.text) if ET.iselement(x:=root.find('nport:formData/nport:fundInfo/nport:cshNotRptdInCorD',ns)) else 0.0
    ret_block = root.findall('nport:formData/nport:fundInfo/nport:returnInfo/nport:monthlyTotReturns/nport:monthlyTotReturn',ns)
    shared_nport_classes = [cid.attrib['classId'] for cid in ret_block if 'classId' in cid.attrib]

    print(f'Filing as of {nport_as_of}\n')
    print(f'Fund Total Assets: {fund_tot_assets}')
    print(f'Fund Total Liabilities: {fund_tot_liabs}')
    print(f'Fund Leverage: {fund_leverage:.2%}')
    print(f'fund cash: ${fund_cash:.2f}')
    print(f'sibling funds found: {shared_nport_classes}\n')
    print('Parsing Holdings...\n')
    print('Unidentified Holdings:')

    # extract holdings info
    ready_for_id = []
    unknown_holding = []
    for holding in root.findall('nport:formData/nport:invstOrSecs/',ns):
        basic = [('isin',x.attrib['value']) if ET.iselement(x:=holding.find('nport:identifiers/nport:isin',ns)) else None,
                 ('ticker',x.attrib['value']) if ET.iselement(x:=holding.find('nport:identifiers/nport:ticker',ns)) else None,
                 (('cusip',x.text) if x.text.isalnum() else None) if ET.iselement(x:=holding.find('nport:cusip',ns)) else None,
                 ('pos_val_usd',float(x.text)) if ET.iselement(x:=holding.find('nport:valUSD',ns)) else None,
                 ('pos_pct_val',round(float(x.text)/100,5)) if ET.iselement(x:=holding.find('nport:pctVal',ns)) else None,
                 ('pos_qty',float(x.text)) if ET.iselement(x:=holding.find('nport:balance',ns)) else None,
                 ('type_qty',x.text) if ET.iselement(x:=holding.find('nport:units',ns)) else None,
                 ('asset_cat',x.text) if ET.iselement(x:=holding.find('nport:assetCat',ns)) else None,
                 ('issuer_cat',x.text) if ET.iselement(x:=holding.find('nport:issuerCat',ns)) else None,
                 (('inv_country',x.text) if x.text.isalnum() else None) if ET.iselement(x:=holding.find('nport:invCountry',ns)) else None,
                 ('currency_info',{'code':x.attrib['curCd'],
                                   'rate': float(x.attrib['exchangeRt'])}) if ET.iselement(x:=holding.find('nport:currencyConditional',ns)) else None]
        holding_info = dict([dp for dp in basic if dp])
        # stocks
        if holding_info['asset_cat'] in ['EC', 'EP'] and holding_info['issuer_cat']=='CORP':
            holding_info['stock_price'] = round(holding_info['pos_val_usd']/holding_info['pos_qty'],2)
            ready_for_id.append(holding_info)
            continue
        # bonds
        if ET.iselement(debt_dps:=holding.find('nport:debtSec',ns)):
            debt_info = {'mat_date':dt.datetime.strptime(x.text,'%Y-%m-%d').date()} if ET.iselement(x:=debt_dps.find('nport:maturityDt',ns)) else {}
            debt_info |= {'cpn_type':x.text} if ET.iselement(x:=debt_dps.find('nport:couponKind',ns)) else {}
            debt_info |= {'cpn_rate':float(x.text)/100} if ET.iselement(x:=debt_dps.find('nport:annualizedRt',ns)) else {}
            holding_info |= debt_info
            ready_for_id.append(holding_info)
            continue
        # for now, just ignore everything else... project for another day
        holding_info |= {'title':x.text} if ET.iselement(x:=holding.find('nport:title',ns)) else {}
        unknown_holding.append(holding_info)
        print(holding_info)
    print('\nSuccessfully parsed {:.2%} of total fund assets'.format(sum([i['pos_val_usd'] for i in ready_for_id])/fund_tot_assets))

    return ready_for_id, unknown_holding



def analyze_holdings(holding_info):
    with open('backend_metadata.json') as file:
        backend_metadata = json.load(file)
    # add geography info in
    holding_info = [h | backend_metadata['geography'][h['inv_country'] if 'inv_country' in h and h['inv_country'] in backend_metadata['geography'] else 'ZZ'] for h in holding_info]

    # identify everything here
    holdings_equity = []
    for h in holding_info:
        if 'asset_cat' in h and h['asset_cat'] in ['EC', 'EP']:
            holdings_equity.append(h)


    # start computing stats
    payload_fund_allocation = sum([e['pos_pct_val'] for e in holdings_equity])
    payload_equity_geography_market = {mkt:round(sum([e['pos_pct_val'] for e in holdings_equity if e['market']==mkt])/payload_fund_allocation,4) 
                                       for mkt in set([h['market'] for h in holdings_equity])}
    payload_equity_geography_region = {rgn:round(sum([e['pos_pct_val'] for e in holdings_equity if e['region']==rgn])/payload_fund_allocation,4)
                                       for rgn in set([h['region'] for h in holdings_equity])}
    payload_equity_geography_country = {cty:round(sum([e['pos_pct_val'] for e in holdings_equity if e['country']==cty])/payload_fund_allocation,4)
                                        for cty in set([h['country'] for h in holdings_equity])}

    print(payload_fund_allocation)
    print(payload_equity_geography_market)
    print(payload_equity_geography_region)
    print(payload_equity_geography_country)

    

    


    return holding_info

def update_equity():


    return


cik = '1100663'
acc_num = '000175272423216302'
good, bad = testing_parse_form_nport(cik, acc_num)
test = analyze_holdings(good)

parsing form nport
https://www.sec.gov/Archives/edgar/data/1100663/000175272423216302/primary_doc.xml
Filing as of 2023-07-31

Fund Total Assets: 2431041460.55
Fund Total Liabilities: 289050.26
Fund Leverage: 100.01%
fund cash: $1138.02
sibling funds found: ['C000217185']

Parsing Holdings...

Unidentified Holdings:
{'isin': 'US0669224778', 'cusip': '066922477', 'pos_val_usd': 10700000.0, 'pos_pct_val': 0.0044, 'pos_qty': 10700000.0, 'type_qty': 'NS', 'asset_cat': 'STIV', 'issuer_cat': 'RF', 'inv_country': 'US', 'title': 'BlackRock Cash Funds: Treasury, SL Agency Shares'}

Successfully parsed 99.11% of total fund assets
0
{}
{}
{}


In [957]:
[(t['mat_date']-dt.date(2023,7,31)).days/365 for t in test if 'mat_date' in t]

[0.6246575342465753,
 0.5835616438356165,
 1.0876712328767124,
 0.7506849315068493,
 1.378082191780822,
 1.336986301369863,
 0.9178082191780822,
 0.7506849315068493,
 1.1287671232876713,
 1.0027397260273974,
 1.0438356164383562,
 0.5452054794520548,
 0.8356164383561644,
 0.5041095890410959,
 1.16986301369863,
 0.8767123287671232,
 0.7095890410958904,
 0.6684931506849315,
 1.295890410958904,
 0.7506849315068493,
 1.295890410958904,
 1.0027397260273974,
 0.958904109589041,
 1.0027397260273974,
 0.4602739726027397]

In [979]:
tot_bond_wgt = sum([t['pos_pct_val'] for t in test])
quote_date = dt.date(2023,7,31)

wgtd_maturity = 0
wgtd_coupon = 0
wgtd_ytm = 0

for bond in test:
    wgt = bond['pos_pct_val']
    maturity = (bond['mat_date']-quote_date).days/365
    coupon = bond['cpn_rate']
    ytm = (coupon+(1-bond['pos_val_usd']/bond['pos_qty'])/(maturity/0.5))/((1+bond['pos_val_usd']/bond['pos_qty'])/2)

    wgtd_maturity += (wgt/tot_bond_wgt)*maturity
    wgtd_coupon += (wgt/tot_bond_wgt)*coupon
    wgtd_ytm += (wgt/tot_bond_wgt)*ytm

print(wgtd_maturity)
print(wgtd_coupon)
print(wgtd_ytm)

0.9284150421934835
0.016760507864125664
0.03512033796814154


In [903]:
geography_map = {'US':{'market': 'Developed','region': 'Americas','country': 'United States'},
                 'CA':{'market': 'Developed','region': 'Americas','country': 'Canada'},
                 'AT':{'market': 'Developed','region': 'Europe','country': 'Austria'},
                 'BE':{'market': 'Developed','region': 'Europe','country': 'Belgium'},
                 'DK':{'market': 'Developed','region': 'Europe','country': 'Denmark'},
                 'FI':{'market': 'Developed','region': 'Europe','country': 'Finland'},
                 'FR':{'market': 'Developed','region': 'Europe','country': 'France'},
                 'DE':{'market': 'Developed','region': 'Europe','country': 'Germany'},
                 'IE':{'market': 'Developed','region': 'Europe','country': 'Ireland'},
                 'IT':{'market': 'Developed','region': 'Europe','country': 'Italy'},
                 'NL':{'market': 'Developed','region': 'Europe','country': 'Netherlands'},
                 'NO':{'market': 'Developed','region': 'Europe','country': 'Norway'},
                 'PT':{'market': 'Developed','region': 'Europe','country': 'Portugal'},
                 'ES':{'market': 'Developed','region': 'Europe','country': 'Spain'},
                 'SE':{'market': 'Developed','region': 'Europe','country': 'Sweden'},
                 'CH':{'market': 'Developed','region': 'Europe','country': 'Switzerland'},
                 'GB':{'market': 'Developed','region': 'Europe','country': 'United Kingdom'},
                 'IL':{'market': 'Developed','region': 'Middle East','country': 'Israel'},
                 'AU':{'market': 'Developed','region': 'Asia-Pacific','country': 'Australia'},
                 'HK':{'market': 'Developed','region': 'Asia-Pacific','country': 'Hong Kong'},
                 'JP':{'market': 'Developed','region': 'Asia-Pacific','country': 'Japan'},
                 'NZ':{'market': 'Developed','region': 'Asia-Pacific','country': 'New Zealand'},
                 'SG':{'market': 'Developed','region': 'Asia-Pacific','country': 'Singapore'},
                 'BR':{'market': 'Emerging','region': 'Americas','country': 'Brazil'},
                 'CL':{'market': 'Emerging','region': 'Americas','country': 'Chile'},
                 'CO':{'market': 'Emerging','region': 'Americas','country': 'Colombia'},
                 'MX':{'market': 'Emerging','region': 'Americas','country': 'Mexico'},
                 'PE':{'market': 'Emerging','region': 'Americas','country': 'Peru'},
                 'CZ':{'market': 'Emerging','region': 'Europe','country': 'Czech Republic'},
                 'GR':{'market': 'Emerging','region': 'Europe','country': 'Greece'},
                 'HU':{'market': 'Emerging','region': 'Europe','country': 'Hungary'},
                 'PO':{'market': 'Emerging','region': 'Europe','country': 'Poland'},
                 'EG':{'market': 'Emerging','region': 'Middle East','country': 'Egypt'},
                 'KW':{'market': 'Emerging','region': 'Middle East','country': 'Kuwait'},
                 'QA':{'market': 'Emerging','region': 'Middle East','country': 'Qatar'},
                 'SA':{'market': 'Emerging','region': 'Middle East','country': 'Saudi Arabia'},
                 'TR':{'market': 'Emerging','region': 'Middle East','country': 'Turkey'},
                 'AE':{'market': 'Emerging','region': 'Middle East','country': 'UAE'},
                 'ZA':{'market': 'Emerging','region': 'Africa','country': 'South Africa'},
                 'CN':{'market': 'Emerging','region': 'Asia-Pacific','country': 'China'},
                 'IN':{'market': 'Emerging','region': 'Asia-Pacific','country': 'India'},
                 'ID':{'market': 'Emerging','region': 'Asia-Pacific','country': 'Indonesia'},
                 'KR':{'market': 'Emerging','region': 'Asia-Pacific','country': 'Korea'},
                 'MY':{'market': 'Emerging','region': 'Asia-Pacific','country': 'Malaysia'},
                 'PH':{'market': 'Emerging','region': 'Asia-Pacific','country': 'Philippines'},
                 'TW':{'market': 'Emerging','region': 'Asia-Pacific','country': 'Taiwan'},
                 'TH':{'market': 'Emerging','region': 'Asia-Pacific','country': 'Thailand'},
                 'AR':{'market': 'Frontier','region': 'Americas','country': 'Argentina'},
                 'JM':{'market': 'Frontier','region': 'Americas','country': 'Jamaica'},
                 'PA':{'market': 'Frontier','region': 'Americas','country': 'Panama'},
                 'TT':{'market': 'Frontier','region': 'Americas','country': 'Trinidad and Tobago'},
                 'HR':{'market': 'Frontier','region': 'Europe','country': 'Croatia'},
                 'EE':{'market': 'Frontier','region': 'Europe','country': 'Estonia'},
                 'IS':{'market': 'Frontier','region': 'Europe','country': 'Iceland'},
                 'LT':{'market': 'Frontier','region': 'Europe','country': 'Lithuania'},
                 'RO':{'market': 'Frontier','region': 'Europe','country': 'Romania'},
                 'RS':{'market': 'Frontier','region': 'Europe','country': 'Serbia'},
                 'SI':{'market': 'Frontier','region': 'Europe','country': 'Slovenia'},
                 'BA':{'market': 'Frontier','region': 'Europe','country': 'Bosnia and Herzegovina'},
                 'BG':{'market': 'Frontier','region': 'Europe','country': 'Bulgaria'},
                 'MT':{'market': 'Frontier','region': 'Europe','country': 'Malta'},
                 'UA':{'market': 'Frontier','region': 'Europe','country': 'Ukraine'},
                 'BH':{'market': 'Frontier','region': 'Middle East','country': 'Bahrain'},
                 'JO':{'market': 'Frontier','region': 'Middle East','country': 'Jordan'},
                 'OM':{'market': 'Frontier','region': 'Middle East','country': 'Oman'},
                 'LB':{'market': 'Frontier','region': 'Middle East','country': 'Lebanon'},
                 'PS':{'market': 'Frontier','region': 'Middle East','country': 'Palestine'},
                 'KZ':{'market': 'Frontier','region': 'Asia-Pacific','country': 'Kazakhstan'},
                 'BD':{'market': 'Frontier','region': 'Asia-Pacific','country': 'Bangladesh'},
                 'PK':{'market': 'Frontier','region': 'Asia-Pacific','country': 'Pakistan'},
                 'LK':{'market': 'Frontier','region': 'Asia-Pacific','country': 'Sri Lanka'},
                 'VN':{'market': 'Frontier','region': 'Asia-Pacific','country': 'Vietnam'},
                 'BJ':{'market': 'Frontier','region': 'Africa','country': 'Benin'},
                 'BF':{'market': 'Frontier','region': 'Africa','country': 'Burkina Faso'},
                 'CI':{'market': 'Frontier','region': 'Africa','country': 'Ivory Coast'},
                 'KE':{'market': 'Frontier','region': 'Africa','country': 'Kenya'},
                 'MU':{'market': 'Frontier','region': 'Africa','country': 'Mauritius'},
                 'MA':{'market': 'Frontier','region': 'Africa','country': 'Morocco'},
                 'NG':{'market': 'Frontier','region': 'Africa','country': 'Nigeria'},
                 'SN':{'market': 'Frontier','region': 'Africa','country': 'Senegal'},
                 'TN':{'market': 'Frontier','region': 'Africa','country': 'Tunisia'},
                 'BW':{'market': 'Frontier','region': 'Africa','country': 'Botswana'},
                 'ZW':{'market': 'Frontier','region': 'Africa','country': 'Zimbabwe'},
                 'ZZ':{'market': 'Other','region': 'Other','country': 'Other'}}

good = [i | geography_map[i['inv_country']if 'inv_country' in i and i['inv_country'] in geography_map else 'ZZ']   for i in good]






nested_sum = lambda d,s,g: {j:sum([k[s] for k in d if k[g[0]]==j]) if len(g)==1 else nested_sum([k for k in d if k[g[0]]==j],s,g[1:]) for j in set([i[g[0]] for i in d])}

nested_sum(good, 'pos_pct_val', ['asset_cat','market','region','country'])

{'EC': {'Emerging': {'Asia-Pacific': {'Korea': 0.016550000000000002,
    'Taiwan': 0.00905,
    'Thailand': 0.00424},
   'Americas': {'Mexico': 0.01398, 'Brazil': 0.01326}},
  'Other': {'Other': {'Other': 0.047389999999999995}},
  'Developed': {'Asia-Pacific': {'Australia': 0.00331,
    'Hong Kong': 0.00109,
    'Japan': 0.06745000000000001},
   'Europe': {'Norway': 0.00253,
    'Ireland': 0.016819999999999998,
    'France': 0.035359999999999996,
    'Belgium': 0.01038,
    'United Kingdom': 0.05949,
    'Germany': 0.00766,
    'Switzerland': 0.030989999999999997,
    'Netherlands': 0.0128,
    'Sweden': 0.00838},
   'Americas': {'Canada': 0.04302, 'United States': 0.36416999999999994}}},
 'STIV': {'Developed': {'Asia-Pacific': {'Singapore': 0.00038},
   'Europe': {'France': 0.018000000000000002,
    'Germany': 0.02266,
    'United Kingdom': 0.00131,
    'Austria': 0.00588,
    'Netherlands': 0.01373},
   'Americas': {'Canada': 0.02686, 'United States': 0.026660000000000007}}},
 'DBT':

In [2]:
import firebase_admin
from firebase_admin import credentials, firestore
cred = credentials.Certificate('creds.json')
firebase_admin.initialize_app(cred)
db = firestore.client()

In [24]:
#test = db.collection('map_figi').document('BBG001T93S65').collection('stats').document('EQTY').collection('biz').document('sector').get()
test = db.document('map_figi/BBG001T93S65/stats/EQTY/biz/industry').get()

In [25]:
test.to_dict()

In [ ]:
# believe it or not, these stupid ass territories actually show up in data...
geography_map = {'US':{'market': 'DM','region': 'AM','country': 'US'},'AS':{'market': 'DM','region': 'AM','country': 'US'},'GU':{'market': 'DM','region': 'AM','country': 'US'}, 
                 'MP':{'market': 'DM','region': 'AM','country': 'US'},'PR':{'market': 'DM','region': 'AM','country': 'US'},'UM':{'market': 'DM','region': 'AM','country': 'US'},
                 'VI':{'market': 'DM','region': 'AM','country': 'US'},'CA':{'market': 'DM','region': 'AM','country': 'CA'},'AT':{'market': 'DM','region': 'EU','country': 'AT'},
                 'BE':{'market': 'DM','region': 'EU','country': 'BE'},'DK':{'market': 'DM','region': 'EU','country': 'DK'},'FO':{'market': 'DM','region': 'EU','country': 'DK'},
                 'GL':{'market': 'DM','region': 'EU','country': 'DK'},'FI':{'market': 'DM','region': 'EU','country': 'FI'},'AX':{'market': 'DM','region': 'EU','country': 'FI'},
                 'FR':{'market': 'DM','region': 'EU','country': 'FR'},'GF':{'market': 'DM','region': 'EU','country': 'FR'},'PF':{'market': 'DM','region': 'EU','country': 'FR'},
                 'TF':{'market': 'DM','region': 'EU','country': 'FR'},'GP':{'market': 'DM','region': 'EU','country': 'FR'},'NC':{'market': 'DM','region': 'EU','country': 'FR'},
                 'BL':{'market': 'DM','region': 'EU','country': 'FR'},'RE':{'market': 'DM','region': 'EU','country': 'FR'},'MF':{'market': 'DM','region': 'EU','country': 'FR'},
                 'PM':{'market': 'DM','region': 'EU','country': 'FR'},'WF':{'market': 'DM','region': 'EU','country': 'FR'},'MQ':{'market': 'DM','region': 'EU','country': 'FR'},
                 'YT':{'market': 'DM','region': 'EU','country': 'FR'},'DE':{'market': 'DM','region': 'EU','country': 'DE'},'IE':{'market': 'DM','region': 'EU','country': 'IE'},
                 'IT':{'market': 'DM','region': 'EU','country': 'IT'},'NL':{'market': 'DM','region': 'EU','country': 'NL'},'AW':{'market': 'DM','region': 'EU','country': 'NL'},
                 'BQ':{'market': 'DM','region': 'EU','country': 'NL'},'SX':{'market': 'DM','region': 'EU','country': 'NL'},'CW':{'market': 'DM','region': 'EU','country': 'NL'},
                 'NO':{'market': 'DM','region': 'EU','country': 'NO'},'BV':{'market': 'DM','region': 'EU','country': 'NO'},'SJ':{'market': 'DM','region': 'EU','country': 'NO'},
                 'PT':{'market': 'DM','region': 'EU','country': 'PT'},'ES':{'market': 'DM','region': 'EU','country': 'ES'},'SE':{'market': 'DM','region': 'EU','country': 'SE'},
                 'CH':{'market': 'DM','region': 'EU','country': 'CH'},'GB':{'market': 'DM','region': 'EU','country': 'GB'},'AI':{'market': 'DM','region': 'EU','country': 'GB'},
                 'IO':{'market': 'DM','region': 'EU','country': 'GB'},'KY':{'market': 'DM','region': 'EU','country': 'GB'},'FK':{'market': 'DM','region': 'EU','country': 'GB'},
                 'GI':{'market': 'DM','region': 'EU','country': 'GB'},'GG':{'market': 'DM','region': 'EU','country': 'GB'},'IM':{'market': 'DM','region': 'EU','country': 'GB'},
                 'JE':{'market': 'DM','region': 'EU','country': 'GB'},'MS':{'market': 'DM','region': 'EU','country': 'GB'},'PN':{'market': 'DM','region': 'EU','country': 'GB'},
                 'SH':{'market': 'DM','region': 'EU','country': 'GB'},'GS':{'market': 'DM','region': 'EU','country': 'GB'},'TC':{'market': 'DM','region': 'EU','country': 'GB'},
                 'BM':{'market': 'DM','region': 'EU','country': 'GB'},'VG':{'market': 'DM','region': 'EU','country': 'GB'},'IL':{'market': 'DM','region': 'ME','country': 'IL'},
                 'AU':{'market': 'DM','region': 'AP','country': 'AU'},'CX':{'market': 'DM','region': 'AP','country': 'AU'},'CC':{'market': 'DM','region': 'AP','country': 'AU'},
                 'HM':{'market': 'DM','region': 'AP','country': 'AU'},'NF':{'market': 'DM','region': 'AP','country': 'AU'},'HK':{'market': 'DM','region': 'AP','country': 'HK'},
                 'JP':{'market': 'DM','region': 'AP','country': 'JP'},'NZ':{'market': 'DM','region': 'AP','country': 'NZ'},'NU':{'market': 'DM','region': 'AP','country': 'NZ'},
                 'TK':{'market': 'DM','region': 'AP','country': 'NZ'},'CK':{'market': 'DM','region': 'AP','country': 'NZ'},'SG':{'market': 'DM','region': 'AP','country': 'SG'},
                 'BR':{'market': 'EM','region': 'AM','country': 'BR'},'CL':{'market': 'EM','region': 'AM','country': 'CL'},'CO':{'market': 'EM','region': 'AM','country': 'CO'},
                 'MX':{'market': 'EM','region': 'AM','country': 'MX'},'PE':{'market': 'EM','region': 'AM','country': 'PE'},'CZ':{'market': 'EM','region': 'EU','country': 'CZ'},
                 'GR':{'market': 'EM','region': 'EU','country': 'GR'},'HU':{'market': 'EM','region': 'EU','country': 'HU'},'PO':{'market': 'EM','region': 'EU','country': 'PO'},
                 'EG':{'market': 'EM','region': 'ME','country': 'EG'},'KW':{'market': 'EM','region': 'ME','country': 'KW'},'QA':{'market': 'EM','region': 'ME','country': 'QA'},
                 'SA':{'market': 'EM','region': 'ME','country': 'SA'},'TR':{'market': 'EM','region': 'ME','country': 'TR'},'AE':{'market': 'EM','region': 'ME','country': 'AE'},
                 'ZA':{'market': 'EM','region': 'AF','country': 'ZA'},'CN':{'market': 'EM','region': 'AP','country': 'CN'},'MO':{'market': 'EM','region': 'AP','country': 'CN'},
                 'IN':{'market': 'EM','region': 'AP','country': 'IN'},'ID':{'market': 'EM','region': 'AP','country': 'ID'},'KR':{'market': 'EM','region': 'AP','country': 'KR'},
                 'MY':{'market': 'EM','region': 'AP','country': 'MY'},'PH':{'market': 'EM','region': 'AP','country': 'PH'},'TW':{'market': 'EM','region': 'AP','country': 'TW'},
                 'TH':{'market': 'EM','region': 'AP','country': 'TH'},'AR':{'market': 'FM','region': 'AM','country': 'AR'},'JM':{'market': 'FM','region': 'AM','country': 'JM'},
                 'PA':{'market': 'FM','region': 'AM','country': 'PA'},'TT':{'market': 'FM','region': 'AM','country': 'TT'},'HR':{'market': 'FM','region': 'EU','country': 'HR'},
                 'EE':{'market': 'FM','region': 'EU','country': 'EE'},'IS':{'market': 'FM','region': 'EU','country': 'IS'},'LT':{'market': 'FM','region': 'EU','country': 'LT'},
                 'RO':{'market': 'FM','region': 'EU','country': 'RO'},'RS':{'market': 'FM','region': 'EU','country': 'RS'},'SI':{'market': 'FM','region': 'EU','country': 'SI'},
                 'BA':{'market': 'FM','region': 'EU','country': 'BA'},'BG':{'market': 'FM','region': 'EU','country': 'BG'},'MT':{'market': 'FM','region': 'EU','country': 'MT'},
                 'UA':{'market': 'FM','region': 'EU','country': 'UA'},'BH':{'market': 'FM','region': 'ME','country': 'BH'},'JO':{'market': 'FM','region': 'ME','country': 'JO'},
                 'OM':{'market': 'FM','region': 'ME','country': 'OM'},'LB':{'market': 'FM','region': 'ME','country': 'LB'},'PS':{'market': 'FM','region': 'ME','country': 'PS'},
                 'KZ':{'market': 'FM','region': 'AP','country': 'KZ'},'BD':{'market': 'FM','region': 'AP','country': 'BD'},'PK':{'market': 'FM','region': 'AP','country': 'PK'},
                 'LK':{'market': 'FM','region': 'AP','country': 'LK'},'VN':{'market': 'FM','region': 'AP','country': 'VN'},'BJ':{'market': 'FM','region': 'AF','country': 'BJ'},
                 'BF':{'market': 'FM','region': 'AF','country': 'BF'},'CI':{'market': 'FM','region': 'AF','country': 'CI'},'KE':{'market': 'FM','region': 'AF','country': 'KE'},
                 'MU':{'market': 'FM','region': 'AF','country': 'MU'},'MA':{'market': 'FM','region': 'AF','country': 'MA'},'NG':{'market': 'FM','region': 'AF','country': 'NG'},
                 'SN':{'market': 'FM','region': 'AF','country': 'SN'},'TN':{'market': 'FM','region': 'AF','country': 'TN'},'BW':{'market': 'FM','region': 'AF','country': 'BW'},
                 'ZW':{'market': 'FM','region': 'AF','country': 'ZW'},'ZZ':{'market': 'ZZ','region': 'ZZ','country': 'ZZ'}}

In [948]:
# believe it or not, these random territories actually show up in data...
geography_map = {'US':{'market': 'DM','region': 'AM','country': 'US'}, # United States
                 'AS':{'market': 'DM','region': 'AM','country': 'US'}, #    American Samoa
                 'GU':{'market': 'DM','region': 'AM','country': 'US'}, #    Guam
                 'MP':{'market': 'DM','region': 'AM','country': 'US'}, #    Northern Mariana Islands
                 'PR':{'market': 'DM','region': 'AM','country': 'US'}, #    Puerto Rico
                 'UM':{'market': 'DM','region': 'AM','country': 'US'}, #    United States Minor Outlying Islands
                 'VI':{'market': 'DM','region': 'AM','country': 'US'}, #    U.S. Virgin Islands
                 'CA':{'market': 'DM','region': 'AM','country': 'CA'}, # Canada
                 'AT':{'market': 'DM','region': 'EU','country': 'AT'}, # Austria
                 'BE':{'market': 'DM','region': 'EU','country': 'BE'}, # Belgium
                 'DK':{'market': 'DM','region': 'EU','country': 'DK'}, # Denmark
                 'FO':{'market': 'DM','region': 'EU','country': 'DK'}, #    Faroe Islands
                 'GL':{'market': 'DM','region': 'EU','country': 'DK'}, #    Greenland
                 'FI':{'market': 'DM','region': 'EU','country': 'FI'}, # Finland
                 'AX':{'market': 'DM','region': 'EU','country': 'FI'}, #    Aland Islands
                 'FR':{'market': 'DM','region': 'EU','country': 'FR'}, # France
                 'GF':{'market': 'DM','region': 'EU','country': 'FR'}, #    French Guiana
                 'PF':{'market': 'DM','region': 'EU','country': 'FR'}, #    French Polynesia
                 'TF':{'market': 'DM','region': 'EU','country': 'FR'}, #    French Southern Territories
                 'GP':{'market': 'DM','region': 'EU','country': 'FR'}, #    Guadeloupe
                 'NC':{'market': 'DM','region': 'EU','country': 'FR'}, #    New Caledonia
                 'BL':{'market': 'DM','region': 'EU','country': 'FR'}, #    Saint Barthelemy
                 'RE':{'market': 'DM','region': 'EU','country': 'FR'}, #    Reunion
                 'MF':{'market': 'DM','region': 'EU','country': 'FR'}, #    Saint Martin
                 'PM':{'market': 'DM','region': 'EU','country': 'FR'}, #    Saint Pierre and Miquelon
                 'WF':{'market': 'DM','region': 'EU','country': 'FR'}, #    Wallis and Futuna
                 'MQ':{'market': 'DM','region': 'EU','country': 'FR'}, #    Martinique
                 'YT':{'market': 'DM','region': 'EU','country': 'FR'}, #    Mayotte
                 'DE':{'market': 'DM','region': 'EU','country': 'DE'}, # Germany
                 'IE':{'market': 'DM','region': 'EU','country': 'IE'}, # Ireland
                 'IT':{'market': 'DM','region': 'EU','country': 'IT'}, # Italy
                 'NL':{'market': 'DM','region': 'EU','country': 'NL'}, # Netherlands
                 'AW':{'market': 'DM','region': 'EU','country': 'NL'}, #    Aruba
                 'BQ':{'market': 'DM','region': 'EU','country': 'NL'}, #    Caribbean Netherlands
                 'SX':{'market': 'DM','region': 'EU','country': 'NL'}, #    Sint Maarten
                 'CW':{'market': 'DM','region': 'EU','country': 'NL'}, #    Curacao
                 'NO':{'market': 'DM','region': 'EU','country': 'NO'}, # Norway
                 'BV':{'market': 'DM','region': 'EU','country': 'NO'}, #    Bouvet Island
                 'SJ':{'market': 'DM','region': 'EU','country': 'NO'}, #    Svalbard and Jan Mayen
                 'PT':{'market': 'DM','region': 'EU','country': 'PT'}, # Portugal 
                 'ES':{'market': 'DM','region': 'EU','country': 'ES'}, # Spain
                 'SE':{'market': 'DM','region': 'EU','country': 'SE'}, # Sweden
                 'CH':{'market': 'DM','region': 'EU','country': 'CH'}, # Switzerland
                 'GB':{'market': 'DM','region': 'EU','country': 'GB'}, # United Kingdom
                 'AI':{'market': 'DM','region': 'EU','country': 'GB'}, #    Anguilla
                 'IO':{'market': 'DM','region': 'EU','country': 'GB'}, #    British Indian Ocean Territory
                 'KY':{'market': 'DM','region': 'EU','country': 'GB'}, #    Cayman Islands
                 'FK':{'market': 'DM','region': 'EU','country': 'GB'}, #    Falkland Islands (Islas Malvinas)
                 'GI':{'market': 'DM','region': 'EU','country': 'GB'}, #    Gibraltar
                 'GG':{'market': 'DM','region': 'EU','country': 'GB'}, #    Guernsey
                 'IM':{'market': 'DM','region': 'EU','country': 'GB'}, #    Isle of Man
                 'JE':{'market': 'DM','region': 'EU','country': 'GB'}, #    Jersey
                 'MS':{'market': 'DM','region': 'EU','country': 'GB'}, #    Montserrat
                 'PN':{'market': 'DM','region': 'EU','country': 'GB'}, #    Pitcairn Islands
                 'SH':{'market': 'DM','region': 'EU','country': 'GB'}, #    Saint Helena, Ascension and Tristan da Cunha
                 'GS':{'market': 'DM','region': 'EU','country': 'GB'}, #    South Georgia and the South Sandwich Islands
                 'TC':{'market': 'DM','region': 'EU','country': 'GB'}, #    Turks and Caicos Islands
                 'BM':{'market': 'DM','region': 'EU','country': 'GB'}, #    Bermuda
                 'VG':{'market': 'DM','region': 'EU','country': 'GB'}, #    British Virgin Islands
                 'IL':{'market': 'DM','region': 'ME','country': 'IL'}, # Israel
                 'AU':{'market': 'DM','region': 'AP','country': 'AU'}, # Australia
                 'CX':{'market': 'DM','region': 'AP','country': 'AU'}, #    Christmas Island
                 'CC':{'market': 'DM','region': 'AP','country': 'AU'}, #    Cocos (Keeling) Islands
                 'HM':{'market': 'DM','region': 'AP','country': 'AU'}, #    Heard and McDonald Islands
                 'NF':{'market': 'DM','region': 'AP','country': 'AU'}, #    Norfolk Island
                 'HK':{'market': 'DM','region': 'AP','country': 'HK'}, # Hong Kong
                 'JP':{'market': 'DM','region': 'AP','country': 'JP'}, # Japan
                 'NZ':{'market': 'DM','region': 'AP','country': 'NZ'}, # New Zealand
                 'NU':{'market': 'DM','region': 'AP','country': 'NZ'}, #    Niue
                 'TK':{'market': 'DM','region': 'AP','country': 'NZ'}, #    Tokelau
                 'CK':{'market': 'DM','region': 'AP','country': 'NZ'}, #    Cook Islands
                 'SG':{'market': 'DM','region': 'AP','country': 'SG'}, # Singapore
                 'BR':{'market': 'EM','region': 'AM','country': 'BR'}, # Brazil
                 'CL':{'market': 'EM','region': 'AM','country': 'CL'}, # Chile
                 'CO':{'market': 'EM','region': 'AM','country': 'CO'}, # Colombia
                 'MX':{'market': 'EM','region': 'AM','country': 'MX'}, # Mexico
                 'PE':{'market': 'EM','region': 'AM','country': 'PE'}, # Peru
                 'CZ':{'market': 'EM','region': 'EU','country': 'CZ'}, # Czech Republic
                 'GR':{'market': 'EM','region': 'EU','country': 'GR'}, # Greece
                 'HU':{'market': 'EM','region': 'EU','country': 'HU'}, # Hungary
                 'PO':{'market': 'EM','region': 'EU','country': 'PO'}, # Poland
                 'EG':{'market': 'EM','region': 'ME','country': 'EG'}, # Eqypt
                 'KW':{'market': 'EM','region': 'ME','country': 'KW'}, # Kuwait
                 'QA':{'market': 'EM','region': 'ME','country': 'QA'}, # Qatar
                 'SA':{'market': 'EM','region': 'ME','country': 'SA'}, # Saudi Arabia
                 'TR':{'market': 'EM','region': 'ME','country': 'TR'}, # Turkey
                 'AE':{'market': 'EM','region': 'ME','country': 'AE'}, # United Arab Emirates
                 'ZA':{'market': 'EM','region': 'AF','country': 'ZA'}, # South Africa
                 'CN':{'market': 'EM','region': 'AP','country': 'CN'}, # China
                 'MO':{'market': 'EM','region': 'AP','country': 'CN'}, #    Macao
                 'IN':{'market': 'EM','region': 'AP','country': 'IN'}, # India
                 'ID':{'market': 'EM','region': 'AP','country': 'ID'}, # Indonesia
                 'KR':{'market': 'EM','region': 'AP','country': 'KR'}, # Korea
                 'MY':{'market': 'EM','region': 'AP','country': 'MY'}, # Malaysia
                 'PH':{'market': 'EM','region': 'AP','country': 'PH'}, # Philippines
                 'TW':{'market': 'EM','region': 'AP','country': 'TW'}, # Taiwan
                 'TH':{'market': 'EM','region': 'AP','country': 'TH'}, # Thailand
                 'AR':{'market': 'FM','region': 'AM','country': 'AR'}, # Argentina
                 'JM':{'market': 'FM','region': 'AM','country': 'JM'}, # Jamaica
                 'PA':{'market': 'FM','region': 'AM','country': 'PA'}, # Panama
                 'TT':{'market': 'FM','region': 'AM','country': 'TT'}, # Trinidad and Tobago
                 'HR':{'market': 'FM','region': 'EU','country': 'HR'}, # Croatia
                 'EE':{'market': 'FM','region': 'EU','country': 'EE'}, # Estonia
                 'IS':{'market': 'FM','region': 'EU','country': 'IS'}, # Iceland
                 'LT':{'market': 'FM','region': 'EU','country': 'LT'}, # Lithuania
                 'RO':{'market': 'FM','region': 'EU','country': 'RO'}, # Romania
                 'RS':{'market': 'FM','region': 'EU','country': 'RS'}, # Serbia
                 'SI':{'market': 'FM','region': 'EU','country': 'SI'}, # Slovenia
                 'BA':{'market': 'FM','region': 'EU','country': 'BA'}, # Bosnia and Herzegovina
                 'BG':{'market': 'FM','region': 'EU','country': 'BG'}, # Bulgaria
                 'MT':{'market': 'FM','region': 'EU','country': 'MT'}, # Malta
                 'UA':{'market': 'FM','region': 'EU','country': 'UA'}, # Ukraine
                 'BH':{'market': 'FM','region': 'ME','country': 'BH'}, # Bahrain
                 'JO':{'market': 'FM','region': 'ME','country': 'JO'}, # Jordan
                 'OM':{'market': 'FM','region': 'ME','country': 'OM'}, # Oman
                 'LB':{'market': 'FM','region': 'ME','country': 'LB'}, # Lebanon
                 'PS':{'market': 'FM','region': 'ME','country': 'PS'}, # Palestine
                 'KZ':{'market': 'FM','region': 'AP','country': 'KZ'}, # Kazakhstan
                 'BD':{'market': 'FM','region': 'AP','country': 'BD'}, # Bangladesh
                 'PK':{'market': 'FM','region': 'AP','country': 'PK'}, # Pakistan
                 'LK':{'market': 'FM','region': 'AP','country': 'LK'}, # Sri Lanka
                 'VN':{'market': 'FM','region': 'AP','country': 'VN'}, # Vietnam
                 'BJ':{'market': 'FM','region': 'AF','country': 'BJ'}, # Benin
                 'BF':{'market': 'FM','region': 'AF','country': 'BF'}, # Burkina Faso
                 'CI':{'market': 'FM','region': 'AF','country': 'CI'}, # Ivory Coast
                 'KE':{'market': 'FM','region': 'AF','country': 'KE'}, # Kenya
                 'MU':{'market': 'FM','region': 'AF','country': 'MU'}, # Mauritius
                 'MA':{'market': 'FM','region': 'AF','country': 'MA'}, # Morocco
                 'NG':{'market': 'FM','region': 'AF','country': 'NG'}, # Nigeria
                 'SN':{'market': 'FM','region': 'AF','country': 'SN'}, # Senegal
                 'TN':{'market': 'FM','region': 'AF','country': 'TN'}, # Tunisia
                 'BW':{'market': 'FM','region': 'AF','country': 'BW'}, # Botswana
                 'ZW':{'market': 'FM','region': 'AF','country': 'ZW'}, # Zimbabwe
                 'ZZ':{'market': 'ZZ','region': 'ZZ','country': 'ZZ'}} # Unknown/Other


with open('backend_metadata.json','w') as outfile:
    outfile.write(json.dumps({'geography':geography_map}))


In [897]:
def nested_sum(data, sum_key, group_keys):
    return {j:sum([k[sum_key] for k in data if k[group_keys[0]]==j]) if len(group_keys)==1 else nested_sum([k for k in data if k[group_keys[0]]==j],sum_key,group_keys[1:]) for j in set([i[group_keys[0]] for i in data])}

#nested_sum(good2, 'pos_pct_val', ['asset_cat','market','region','country'])
nested_sum(good2, 'pos_pct_val', ['asset_cat','market','region','country'])

{'EC': {'Emerging': {'Asia-Pacific': {'Korea': 0.016550000000000002,
    'Taiwan': 0.00905,
    'Thailand': 0.00424},
   'Americas': {'Mexico': 0.01398, 'Brazil': 0.01326}},
  'Other': {'Other': {'Other': 0.047389999999999995}},
  'Developed': {'Asia-Pacific': {'Australia': 0.00331,
    'Hong Kong': 0.00109,
    'Japan': 0.06745000000000001},
   'Europe': {'Norway': 0.00253,
    'Ireland': 0.016819999999999998,
    'France': 0.035359999999999996,
    'Belgium': 0.01038,
    'United Kingdom': 0.05949,
    'Germany': 0.00766,
    'Switzerland': 0.030989999999999997,
    'Netherlands': 0.0128,
    'Sweden': 0.00838},
   'Americas': {'Canada': 0.04302, 'United States': 0.36416999999999994}}},
 'STIV': {'Developed': {'Asia-Pacific': {'Singapore': 0.00038},
   'Europe': {'France': 0.018000000000000002,
    'Germany': 0.02266,
    'United Kingdom': 0.00131,
    'Austria': 0.00588,
    'Netherlands': 0.01373},
   'Americas': {'Canada': 0.02686, 'United States': 0.026660000000000007}}},
 'DBT':

In [733]:
raw_str


# extract fund (series) info
nport_as_of = dt.datetime.strptime(root.find('nport:formData/nport:genInfo/nport:repPdDate', ns).text,'%Y-%m-%d').date()
fund_tot_assets = float(root.find('nport:formData/nport:fundInfo/nport:totAssets',ns).text)
fund_tot_liabs = float(root.find('nport:formData/nport:fundInfo/nport:totLiabs',ns).text)
fund_leverage = fund_tot_assets / (fund_tot_assets - fund_tot_liabs)
# many data points are optional, or conditional mandatory, so it is good to check it actually exists before Nonetype errors
fund_cash = float(x.text) if ET.iselement(x:=root.find('nport:formData/nport:fundInfo/nport:cshNotRptdInCorD',ns)) else 0.0
ret_block = root.findall('nport:formData/nport:fundInfo/nport:returnInfo/nport:monthlyTotReturns/nport:monthlyTotReturn',ns)
shared_nport_classes = [cid.attrib['classId'] for cid in ret_block if 'classId' in cid.attrib]

print(f'Filing as of {nport_as_of}\n')
print(f'Fund Total Assets: {fund_tot_assets}')
print(f'Fund Total Liabilities: {fund_tot_liabs}')
print(f'Fund Leverage: {fund_leverage:.2%}')
print(f'fund cash: ${fund_cash:.2f}')
print(f'sibling funds found: {shared_nport_classes}\n')
print('Parsing Holdings...\n')
print('Unidentified Holdings:')

ready_for_id = []
for holding in root.findall('nport:formData/nport:invstOrSecs/',ns):
    basic = [('isin',x.attrib['value']) if ET.iselement(x:=holding.find('nport:identifiers/nport:isin',ns)) else None,
             ('ticker',x.attrib['value']) if ET.iselement(x:=holding.find('nport:identifiers/nport:ticker',ns)) else None,
             (('cusip',x.text) if x.text.isalnum() else None) if ET.iselement(x:=holding.find('nport:cusip',ns)) else None,
             ('pos_val_usd',float(x.text)) if ET.iselement(x:=holding.find('nport:valUSD',ns)) else None,
             ('pos_pct_val',round(float(x.text)/100,5)) if ET.iselement(x:=holding.find('nport:pctVal',ns)) else None,
             ('pos_qty',float(x.text)) if ET.iselement(x:=holding.find('nport:balance',ns)) else None,
             ('type_qty',x.text) if ET.iselement(x:=holding.find('nport:units',ns)) else None,
             ('asset_cat',x.text) if ET.iselement(x:=holding.find('nport:assetCat',ns)) else None,
             ('issuer_cat',x.text) if ET.iselement(x:=holding.find('nport:issuerCat',ns)) else None,
             (('inv_country',x.text) if x.text.isalnum() else None) if ET.iselement(x:=holding.find('nport:invCountry',ns)) else None,
             ('currency_info',{'code':x.attrib['curCd'],
                               'rate': float(x.attrib['exchangeRt'])}) if ET.iselement(x:=holding.find('nport:currencyConditional',ns)) else None]
    holding_info = dict([dp for dp in basic if dp])
    # identify stocks
    if holding_info['asset_cat'] in ['EC', 'EP'] and holding_info['issuer_cat']=='CORP':
        holding_info['stock_price'] = round(holding_info['pos_val_usd']/holding_info['pos_qty'],2)
        ready_for_id.append(holding_info)
        continue
    if ET.iselement(debt_dps:=holding.find('nport:debtSec',ns)):
        debt_info = {'mat_date':dt.datetime.strptime(x.text,'%Y-%m-%d').date()} if ET.iselement(x:=debt_dps.find('nport:maturityDt',ns)) else {}
        debt_info |= {'cpn_type':x.text} if ET.iselement(x:=debt_dps.find('nport:couponKind',ns)) else {}
        debt_info |= {'cpn_rate':float(x.text)/100} if ET.iselement(x:=debt_dps.find('nport:annualizedRt',ns)) else {}
        holding_info |= debt_info
        ready_for_id.append(holding_info)
        continue
    holding_info |= {'title':x.text} if ET.iselement(x:=holding.find('nport:title',ns)) else {}
    print(holding_info)
print('\nSuccessfully parsed {:.2%} of total fund assets'.format(sum([i['pos_val_usd'] for i in ready_for_id])/fund_tot_assets))

Filing as of 2023-07-31

Fund Total Assets: 50506687151.17
Fund Total Liabilities: 67241916.26
Fund Leverage: 100.13%
fund cash: $301.24
sibling funds found: ['C000179763', 'C000030894', 'C000179765', 'C000179764', 'C000030895', 'C000179766', 'C000030896']

Parsing Holdings...

Unidentified Holdings:
{'pos_val_usd': 2497696.58, 'pos_pct_val': 5e-05, 'pos_qty': 1.0, 'type_qty': 'NC', 'asset_cat': 'DFE', 'title': 'Forward Foreign Currency Contract'}
{'pos_val_usd': 2555067.54, 'pos_pct_val': 5e-05, 'pos_qty': 1.0, 'type_qty': 'NC', 'asset_cat': 'DFE', 'title': 'Forward Foreign Currency Contract'}
{'pos_val_usd': 1988155.08, 'pos_pct_val': 4e-05, 'pos_qty': 1.0, 'type_qty': 'NC', 'asset_cat': 'DFE', 'title': 'Forward Foreign Currency Contract'}
{'pos_val_usd': 1544409.86, 'pos_pct_val': 3e-05, 'pos_qty': 1.0, 'type_qty': 'NC', 'asset_cat': 'DFE', 'title': 'Forward Foreign Currency Contract'}
{'pos_val_usd': 417547.75, 'pos_pct_val': 1e-05, 'pos_qty': 1.0, 'type_qty': 'NC', 'asset_cat': 'D

In [720]:
ready_for_id

[{'isin': 'US9100471096',
  'cusip': '910047109',
  'pos_val_usd': 428463423.87,
  'pos_pct_val': 0.00048,
  'pos_qty': 7808701.0,
  'type_qty': 'NS',
  'asset_cat': 'EC',
  'issuer_cat': 'CORP',
  'inv_country': 'US',
  'stock_price': 54.87},
 {'isin': 'US2538681030',
  'cusip': '253868103',
  'pos_val_usd': 789859141.13,
  'pos_pct_val': 0.00089,
  'pos_qty': 6936499.0,
  'type_qty': 'NS',
  'asset_cat': 'EC',
  'issuer_cat': 'CORP',
  'inv_country': 'US',
  'stock_price': 113.87},
 {'isin': 'US37045V1008',
  'cusip': '37045V100',
  'pos_val_usd': 1276267440.32,
  'pos_pct_val': 0.00144,
  'pos_qty': 33098222.0,
  'type_qty': 'NS',
  'asset_cat': 'EC',
  'issuer_cat': 'CORP',
  'inv_country': 'US',
  'stock_price': 38.56},
 {'isin': 'US2786421030',
  'cusip': '278642103',
  'pos_val_usd': 568988655.69,
  'pos_pct_val': 0.00064,
  'pos_qty': 12731901.0,
  'type_qty': 'NS',
  'asset_cat': 'EC',
  'issuer_cat': 'CORP',
  'inv_country': 'US',
  'stock_price': 44.69},
 {'isin': 'US43300A2

In [111]:
# generic function to search for company submissions at the SEC, return the filing id (accession number)
def get_latest_filing(sec_id, submission_type):
    # submission_type:
    #   NPORT-P = most recent portfolio investments report
    #   N-CEN   = most recent annual report for registered investment companies
    submissions_url = f'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={sec_id}&type={submission_type}&output=atom' 
    print(submissions_url)
    submissions_header = # removed personal info
    submissions_response = requests.get(url=submissions_url, headers=submissions_header)
    if submissions_response.status_code != 200:
        return None
    raw_str = submissions_response.content.decode()
    ns = {'subs': raw_str.split('xmlns="')[1].split('"')[0]}
    root = ET.fromstring(raw_str)
    return filing_no.text.replace('-','') if ET.iselement(filing_no:=root.find('subs:entry/subs:content/subs:accession-number', ns)) else None





def get_figi(sec_list): 
    print('called figi')
    # right now can only identify by ISIN, which is very common, and only for equities in current state of testing
    all_jobs = [{'idType':'ID_ISIN', 'idValue':s['isin'], 'marketSecDes':'Equity'} for s in sec_list if ('isin' in s) and (s['stype']=='EQTY')]
    jobs_per_req = 10
    wait_per_req = 2.5
    openfigi_url = 'https://api.openfigi.com/v3/mapping'
    openfigi_headers = {'Content-Type': 'text/json'}
    # TO DO : get an api key so I can go faster
    out_data = {}
    i = 0
    while i<len(all_jobs)/jobs_per_req:
        job_slice = all_jobs[i*jobs_per_req:(i+1)*jobs_per_req]
        time.sleep(0 if i==0 else wait_per_req-min((dt.datetime.now()-last_call).seconds,wait_per_req))
        response = requests.post(url=openfigi_url, headers=openfigi_headers, json=job_slice)
        last_call = dt.datetime.now()
        if response.status_code == 200:
            for input, output in zip(job_slice, response.json()):
                if 'data' not in output:
                    continue 
                figi = list(set([f['shareClassFIGI'] for f in output['data']]))
                if len(figi)==1:
                    out_data[figi[0]] = [s for s in sec_list if s['isin']==input['idValue']][0]
        else:
            print(response.status_code)
            print(response.content)
        i += 1
    return out_data



def parse_form_ncen(cik, acc_num):
    ncen_url = f'https://www.sec.gov/Archives/edgar/data/{cik}/{acc_num}/primary_doc.xml'
    print(ncen_url)
    return




def parse_form_nport(cik, acc_num):
    nport_url = f'https://www.sec.gov/Archives/edgar/data/{cik}/{acc_num}/primary_doc.xml'
    print(nport_url)
    nport_header = # removed personal info
    nport_response = requests.get(url=nport_url, headers=nport_header)
    if nport_response.status_code != 200:
        return None
    raw_str = nport_response.content.decode()
    ns = {'nport': raw_str.split('xmlns="')[1].split('"')[0]}
    root = ET.fromstring(raw_str)
    nport_as_of = dt.datetime.strptime(root.find('nport:formData/nport:genInfo/nport:repPdDate', ns).text,'%Y-%m-%d').date()
    stype_map = {'STIV': None, # short-term investment vehicle
                 'RA': None, # repurchase agreement
                 'EC': 'EQTY', # equity, common
                 'EP': 'EQTY', # equity, preferred
                 'DBT': None, # debt
                 'DCO': None, # derivatives, commodity 
                 'DCR': None, # derivatives, credit
                 'DE': None, # derivatives, equity
                 'DFE': None, # derivatives, foreign exchange
                 'DIR': None, #derivatives, interest rate
                 'DO': None, # derivatives, other
                 'SN': None, # structured note
                 'LON': None, # loan
                 'ABS-MBS': None, # ABS, mortgage-backed security
                 'ABS-APCP': None, # ABS, asset-backed commercial paper
                 'ABS-CBDO': None, # ABS, collaterilized bond/debt obligation
                 'ABS-O': None, # ABS, other
                 'COMM': None, # commodity
                 'RE': None} # real estate
    holdings = []
    for h in root.findall('nport:formData/nport:invstOrSecs/', ns):
        holding_info = {}
        if ET.iselement(isin:=h.find('nport:identifiers/nport:isin', ns)):
            holding_info['isin'] = isin.attrib['value']
        # eventually, add other valid identifiers
        asset_class = h.find('nport:assetCat', ns).text
        usd_value = float(h.find('nport:valUSD', ns).text)
        pct_value = float(h.find('nport:pctVal', ns).text)/100
        inv_country = h.find('nport:invCountry', ns).text # do something with regions, developed vs emerging vs frontier

        holding_info['stype'] = stype_map[asset_class] if asset_class in stype_map else None
        holding_info['stype'] = 'FUND' if h.find('nport:issuerCat').text in ['PF', 'RF'] else holding_info['stype'] # fix the security type for funds

        holding_info['wgt'] = pct_value


        if holding_info['stype']=='EQTY':
            num_shares = float(h.find('nport:balance', ns).text)
            share_price = round(usd_value/num_shares,2)
            holding_info['mark_price'] = share_price
            holding_info['mark_date'] = nport_as_of

        holdings.append(holding_info)
    return analyze_holdings(holdings)






def analyze_holdings(holdings):
    known_holdings = {}
    unknown_holdings = []
    for h in holdings:
        if 'isin' in h:
            figi = db.collection('map_isin').document(h['isin']).get()
            if figi.exists:
                known_holdings[figi.to_dict()['figi']] = h
            else:
                unknown_holdings.append(h)
    identified_holdings = known_holdings | get_figi(unknown_holdings)
    for holding, info in identified_holdings.items():
        if info['stype']=='EQTY':
            curr_data = db.collection('map_figi').document(holding).get()
            if curr_data.exists:
                update_equity(holding, data= info | curr_data.to_dict())
            else:
                print('pending review', holding, info)
                
    return identified_holdings





def update_equity(figi, data=None):
    # TO DO: check if equity has been updated recently, and if so return current data
    print(f'update_equity: {figi}')
    payload = {'stype':'EQTY'}
    stats_payload = {}
    facts_url = f'https://data.sec.gov/api/xbrl/companyfacts/CIK{data["cik"].zfill(10)}.json'
    print(facts_url)
    facts_header = # removed personal info
    facts_response = requests.get(url=facts_url, headers=facts_header)
    if facts_response.status_code != 200:
        print('bad response', facts_response.status_code, facts_response.content)
        return
    response_dict = json.loads(facts_response.content.decode())
    payload['name'] = response_dict['entityName']
    facts = response_dict['facts']

    shares_outstanding = None
    market_cap = None

    time.sleep(1) # simulates all the gaap parsing and keeps the SEC happy


    # updating the firestore db 
    if ('isin' in data) and (not db.collection('map_isin').document(data['isin']).get().exists):
        db.collection('map_isin').document(data['isin']).set({'figi':figi})
    db.collection('map_figi').document(figi).set(payload, merge=True)

    # updating the eq_stats sub-collection
    if market_cap:
        stats_payload['market_cap'] = market_cap
    #db.collection('map_figi').document(figi).collection('stats').document('eqty_stats').set(stats_payload, merge=True)
    return





def update_fund(figi, curr_data, sdata=None, do_ncen=False, do_nport=True):
    print(f'running update_fund on {figi}')
    print(curr_data)
    primary_id = curr_data['sid'] if 'sid' in curr_data else curr_data['cik']

    if do_ncen:
        ncen_accession_number = get_latest_filing(primary_id, 'N-CEN')
        if ncen_accession_number:
            parse_form_ncen(curr_data['cik'], ncen_accession_number)
    if do_nport:
        nport_accession_number = get_latest_filing(primary_id, 'NPORT-P')
        if nport_accession_number:
            fund_stats = parse_form_nport(curr_data['cik'], nport_accession_number)

    return fund_stats





# main entry point
#@functions_framework.cloud_event
def backend(cloud_event):
    try:
        pubsub_data = json.loads(base64.b64decode(cloud_event.data["message"]["data"]).decode('UTF-8'))
    except:
        print('failed at json load')
        return
    
    # pubsub_data dict should have the 'symbol' key at least (which should be a figi at this point), but may have 'stype' and 'sdata' keys
    # stype prevents a figi call just to see which path we're going down if we already know a good identifier... normally we'll have this info when calling this function
    # sdata is a place to throw additional information encountered while seeing the security we're updating
    #     e.g. share price at some date for equities, or maturity & coupon for bonds... depends on the stype what we get
    if 'symbol' not in pubsub_data:
        return
    current_data = db.collection('map_figi').document(pubsub_data['symbol']).get()
    if not current_data.exists:
        print(f'could not find {pubsub_data["symbol"]} in map_figi')
        return

    # check if it's necessary to update 
    #stale_data_threshold = 90 # days
    #try:
        #proceed_update = (dt.date.today()-dt.datetime.strptime(current_data['as_of'], '%Y%m%d').date()).days > stale_data_threshold
    #except:
    #    proceed_update = True
    #if not proceed_update:
    #    print('data up to date')
    #    return

    if 'stype' not in pubsub_data:
        # at this point call figi to see if it can identify the security type... use of this will fade over time as the database gets updated
        # for now, just return
        print('unknown security type')
        return
    if pubsub_data['stype'] == 'FUND':
        print('going fund route')
        update_fund(pubsub_data['symbol'], current_data.to_dict())
    elif pubsub_data['stype'] == 'EQTY':
        print('going equity route')
        update_equity(pubsub_data['symbol'])
    #elif pubsub_data['stype'] == 'BOND':
    #    print('going bond route')
    #    update_bond(pubsub_data['symbol'])

https://www.sec.gov/Archives/edgar/data/1388485/000175272423220151/primary_doc.xml
called figi
update_equity: BBG001S5N9M6
https://data.sec.gov/api/xbrl/companyfacts/CIK0000001800.json
update_equity: BBG009S3NB21
https://data.sec.gov/api/xbrl/companyfacts/CIK0001652044.json
update_equity: BBG001S5PQL7
https://data.sec.gov/api/xbrl/companyfacts/CIK0001018724.json
update_equity: BBG001SKNNS6
https://data.sec.gov/api/xbrl/companyfacts/CIK0001141391.json
update_equity: BBG001S5SD60
https://data.sec.gov/api/xbrl/companyfacts/CIK0000749251.json
update_equity: BBG001S5TD05
https://data.sec.gov/api/xbrl/companyfacts/CIK0000789019.json
update_equity: BBG001SF4NY1
https://data.sec.gov/api/xbrl/companyfacts/CIK0001110803.json
update_equity: BBG001T4JFC0
https://data.sec.gov/api/xbrl/companyfacts/CIK0001373715.json
update_equity: BBG001SF6L46
https://data.sec.gov/api/xbrl/companyfacts/CIK0001065280.json
update_equity: BBG0039320P7
https://data.sec.gov/api/xbrl/companyfacts/CIK0001555280.json
updat

{'BBG001S5N9M6': {'isin': 'US0028241000',
  'stype': 'EQ',
  'wgt': 0.035239410201160004,
  'mark_price': 111.33,
  'mark_date': datetime.date(2023, 7, 31)},
 'BBG009S3NB21': {'isin': 'US02079K1079',
  'stype': 'EQ',
  'wgt': 0.06898494552751,
  'mark_price': 133.11,
  'mark_date': datetime.date(2023, 7, 31)},
 'BBG001S5PQL7': {'isin': 'US0231351067',
  'stype': 'EQ',
  'wgt': 0.1373093955531,
  'mark_price': 133.68,
  'mark_date': datetime.date(2023, 7, 31)},
 'BBG001SKNNS6': {'isin': 'US57636Q1040',
  'stype': 'EQ',
  'wgt': 0.045746451253069996,
  'mark_price': 394.28,
  'mark_date': datetime.date(2023, 7, 31)},
 'BBG001S5SD60': {'isin': 'US3666511072',
  'stype': 'EQ',
  'wgt': 0.01883321435265,
  'mark_price': 353.59,
  'mark_date': datetime.date(2023, 7, 31)},
 'BBG001S5TD05': {'isin': 'US5949181045',
  'stype': 'EQ',
  'wgt': 0.05669932931177,
  'mark_price': 335.92,
  'mark_date': datetime.date(2023, 7, 31)},
 'BBG001SF4NY1': {'isin': 'US4523271090',
  'stype': 'EQ',
  'wgt': 0